In [16]:
#Get wikipedia contents
from nltk.stem.porter import *
import itertools
import os
import string
import wikipedia as wiki

def get_wikipedia_content_based_on_keyword_list(keywordlist, output_save_path):
    os.chdir(output_save_path)
    lst_keyword = keywordlist
    n_total = len(lst_keyword)
    file = open('wikioutputcombined.txt', 'w')
    for index, line in enumerate(lst_keyword):
        keyword = line.strip('\n').lower()
        print index, n_total, index * 1.0 / n_total, keyword
        try:
            content = wiki.page(keyword).content.encode('ascii', 'ignore')
        except wiki.exceptions.DisambiguationError as e:
            print 'DisambiguationError', keyword
        except:
            print 'Error', keyword
        if not content:
            continue
        for line in content.split('\n'):
            line = ' '.join(map(norm_word, line.split()))
            if line:
                file.write(line + '\n')
    file.close()


def get_wikipedia_content_based_on_keyword_list_one_file_per_keyword(keywordlist, output_save_path):
    os.chdir(output_save_path)
    lst_keyword = keywordlist
    n_total = len(lst_keyword)
    for index, line in enumerate(lst_keyword):
        keyword = line.strip('\n').lower()
        print index, n_total, index * 1.0 / n_total, keyword
        try:
            content = wiki.page(keyword).content.encode('ascii', 'ignore')
        except wiki.exceptions.DisambiguationError as e:
            print 'DisambiguationError', keyword
        except:
            print 'Error', keyword
        if not content:
            continue
        file = open('_'.join(keyword.split()) + '.txt', 'w')
        for line in content.split('\n'):
            line = ' '.join(map(norm_word, line.split()))
            if line:
                file.write(line + '\n')
        file.close()

def norm_word(word):
    word = word.lower().strip('?').strip('.').strip(',').strip('!').strip(':').strip(';').strip('\"').strip('\'')
    return word 

In [17]:
import nltk
from nltk.corpus import wordnet
import gensim
import logging
import os
import pandas as pd
import numpy as np
from scipy import spatial
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora
import re

class MyDocument(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            with open(os.path.join(self.dirname, fname)) as content_file:
                content = content_file.read()  
                content = unicode(content, encoding='utf-8', errors='replace')
                yield content.lower().split()

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                line = unicode(line, encoding='utf-8', errors='replace')
                yield line.lower().split()

def get_dictionary(path):
    dictionary = corpora.Dictionary( MySentences('/Users/MK/GitHub/the_answer_is/data/temporary2') )
    stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
    dictionary.filter_tokens(stop_ids) # remove stop words and words that appear only once
    dictionary.compactify() # remove gaps in id sequence after words that were removed
    return dictionary

def get_document_tuple( path, dictionary ):
    vector = pd.Series()
    doclist = MyDocument(path) 
    for i,content in enumerate(doclist):
        vector = vector.set_value(i,dictionary.doc2bow(content))
    return list(vector)

def transform_tuple_into_vector(document_tuple,dictionary):
    vector = pd.Series(np.zeros(len(dictionary.token2id)))        #0 array for storing wiki document vectors.
    if not document_tuple == []:
        for onetuple in document_tuple: 
            vector[onetuple[0]] = onetuple[1]
        vector = vector / np.linalg.norm(vector, ord = 1)                       #normalize vector     
        return vector 
    else:
        return vector

def transform_tuples_into_dataframe(document_tuples, dictionary):
    length = len(document_tuples) 
    vector_initialization = pd.Series(np.zeros(len(dictionary.token2id)))  #0 array for storing wiki document vectors. 
    df_vector = pd.DataFrame(vector_initialization)  #initialize dataframe. all vectors will be stored. 
    for i in xrange(len(document_tuples)):           
        #for each wiki documents, we will transform wiki vectors in tuple form into
        #vectors in ususal form. 
        vector = transform_tuple_into_vector( document_tuples[i], dictionary )
        df_vector[i]= pd.DataFrame(vector)
    return df_vector

def get_close_documents(string, dataframe, dictionary, topn):
    string = unicode(string, encoding='utf-8', errors='replace').lower()
    first_vector = transform_tuple_into_vector(dictionary.doc2bow(string.split()), dictionary )
    lengthlist = pd.Series()
    for j in xrange(len(dataframe.columns)):
        #lengthlist = lengthlist.set_value(j, np.linalg.norm(first_vector-dataframe[j]) )
        lengthlist = lengthlist.set_value( j, spatial.distance.cosine(first_vector, dataframe[j]))
    return lengthlist.sort_values().head(topn)

def get_document_by_index(path,index):
    # this gets the filename and content of the document in a directory by index. 
    i= 0
    for fname in os.listdir(path):
        if i == index:
            with open(os.path.join(path, fname)) as content_file:
                content = content_file.read()  
                content = unicode(content, encoding='utf-8', errors='replace')
                return fname, content
        i= i + 1
    return None

def print_names_of_close_documents(path, close_documents_list, outputpath):
    file = open(outputpath + '/close_documents.txt', 'w+')
    for i in close_documents_list.index:
        name, text = get_document_by_index(path,i)
        print name
        file.write(text + '\n' + '\n' + '\n' + '\n')        
    print 'text saved as txt'

def get_contents_of_close_documents_by_paragraph(path, close_documents_list):
    merged = ''
    for i in close_documents_list.index:
        name, text = get_document_by_index(path,i)
        merged = merged + '\n' + text        
    return merged.split('\n')


def get_distance_between_two_documents(A, B, dictionary):
    import re
    if A:
        A = re.sub(r'[^a-zA-Z ]',r'',A).lower().split()
    else:
        A = ['the']        # just in case A, or B is empty. 
    A = dictionary.doc2bow(A)
    A = transform_tuple_into_vector( A ,dictionary)
    
    if B:
        B = re.sub(r'[^a-zA-Z ]',r'',B).lower().split()
    else:
        B = ['the']       # just in case A, or B is empty. 
    B = dictionary.doc2bow(B)
    B = transform_tuple_into_vector( B ,dictionary)
    #length = spatial.distance.cosine(A,B)
    # I would like to use cosine distance, but the vectors are so sparse that most of the time the output is 0 .
    # So we use euclidean distance. 
    length = np.linalg.norm(A-B)
    return length


stoplist = [u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours',
             u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', 
             u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', 
             u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', 
             u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', 
             u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', 
             u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', 
             u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', 
             u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', 
             u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', 
             u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', 
             u'should', u'now']


#procedure needed for deleting words not in the training set. 
def only_the_words_in_index( list, index ):
    output = []
    for a, s in enumerate(list):
        if s in index:
            output.append(list[a])
    return output

def split_paragraph_list(paragraph_list):
    output = []
    for paragraph in paragraph_list:
        output.append( paragraph.split() )
    return output


def getdef(word):
    '''
    :param word: input is a word in a string format
    :return: returns a string that combines 3 definitions. 
    '''
    #addef = word.decode("utf8",'ignore')
    addef = word            #the word itself goes into the definition.

    for synset in wordnet.synsets(word)[:3]:
        try:
            addef = addef + ' ' + synset.definition()
        except:
            continue
    return addef

def doc_to_def(doc):
    ''' 
    :param doc: input is a document in a string format. Ex: "I have an apple" 
    :return: returns a list of definitions. The length of the list is the number of words in a string.   
    '''
    definition = []
    tokens = nltk.word_tokenize(doc)

    length = len(tokens)    # for printing out the progress
    #print 'doc_to_def progress:  0 / ', length
    progress = 0

    for element in tokens:
        temp = getdef(element)
        definition.append(temp)

        progress = progress + 1
        #print 'doc_to_def progress: ', progress, ' / ', length

    return definition

def split_definition(definition):
    split_def = []
    for index, line in enumerate(definition):
        line = line.lower()
        try:
            line = re.sub(r'[^a-zA-Z ]',r'',line)        #delete all but alphabet 
        except:
            print 'Error', line
        if not line:
            continue

        if line:                              # delete the empty lines. 
            line = line.split()
            #line = nltk.word_tokenize(line)
            line = [word for word in line if word not in stoplist]  # remove the stop words. 
            if line:
                split_def = split_def + line

    return split_def




In [18]:
dictionary_path = '/Users/MK/GitHub/the_answer_is/data/temporary'  # question text from training data
dictionary = get_dictionary(dictionary_path)
#keywordlist = list(dictionary.token2id)
#print keywordlist[0:1400]
keywordlist1 = [u'xylem', u'limited', u'similarity', u'phenotypical', u'alleles', u'magnetic', u'saves', u'desirable', u'crumpled', u'yellow', u'chaos', u'four', u'tensile', u'heliocentric', u'controversial', u'consists', u'oldest', u'worked', u'aggression', u'poorly', u'relationships', u'whose', u'fronts', u'pedro', u'calculate', u'plumes', u'electricity', u'powdery', u'metamorphic', u'seriously', u'supported', u'list', u'strawberries', u'investigation', u'swap', u'caused', u'recycle', u'herbicide', u'shepherd', u'quartzite', u'divergent', u'humerus', u'hormone', u'risk', u'downstream', u'geology', u'barium', u'void', u'oceans', u'connects', u'pigment', u'replication', u'every', u'jack', u'peripheral', u'formula', u'overcomes', u'vastly', u'incubated', u'vast', u'anemia', u'blackcoated', u'rainfalls', u'school', u'baking', u'parrot', u'investigator', u'crops', u'cause', u'solution', u'humidity', u'solid', u'fossilized', u'red', u'succession', u'divides', u'venus', u'chemically', u'reliable', u'methane', u'chloroplast', u'originate', u'force', u'cooperative', u'concise', u'miller', u'consistent', u'bubbling', u'feathers', u'direct', u'tires', u'surrounding', u'likely', u'street', u'chen', u'arms', u'estimated', u'machines', u'shining', u'blue', u'established', u'settlement', u'previously', u'organisms', u'nonfunctioning', u'panama', u'bicarbonate', u'contributed', u'lizard', u'malleable', u'lights', u'glands', u'fallow', u'localized', u'establishes', u'designing', u'new', u'net', u'increasing', u'ever', u'consisting', u'told', u'evolving', u'barometer', u'sinks', u'lithosphere', u'mechanisms', u'hydrangea', u'never', u'drew', u'atoms', u'met', u'feeding', u'songbirds', u'cart', u'ethanol', u'represented', u'watersoluble', u'entire', u'cardboard', u'interpret', u'contraction', u'obtained', u'substance', u'daughter', u'uplifting', u'items', u'study', u'changed', u'reports', u'tolerance', u'jogging', u'ethane', u'launched', u'smoke', u'browsing', u'calculators', u'military', u'melaleuca', u'suitable', u'settled', u'pairings', u'diameter', u'topography', u'projection', u'classification', u'explained', u'kiloliters', u'seven', u'highly', u'neutrons', u'visible', u'pyramid', u'prokaryotes', u'counts', u'total', u'unit', u'redflowered', u'dna', u'spoke', u'would', u'alligator', u'marine', u'sweater', u'chooses', u'diaphragm', u'call', u'bedrock', u'recommend', u'strike', u'survive', u'retriever', u'type', u'tell', u'lightyear', u'elias', u'separated', u'females', u'lungs', u'populations', u'composite', u'expose', u'pancreatic', u'meteorologist', u'disruption', u'warm', u'adult', u'separates', u'containerswhich', u'organic', u'hole', u'hold', u'outdated', u'must', u'abundance', u'intake', u'word', u'room', u'equilibrium', u'player', u'ml', u'setup', u'work', u'eclipse', u'worm', u'roof', u'theories', u'talked', u'iridium', u'trilobites', u'temperatures', u'ms', u'blooming', u'elbow', u'ernest', u'sets', u'root', u'example', u'shook', u'cyclewhich', u'indicated', u'give', u'florida', u'climax', u'india', u'naturally', u'sick', u'involve', u'indicates', u'foods', u'woman', u'want', u'supernova', u'appearance', u'times', u'attract', u'absolute', u'tornadoes', u'end', u'regrowth', u'initiate', u'provide', u'ie', u'travel', u'sitting', u'colds', u'damage', u'astronomers', u'faults', u'hot', u'toxin', u'significance', u'answer', u'cheetah', u'ancestor', u'beach', u'aurora', u'classify', u'classifying', u'regional', u'description', u'undergoing', u'replacing', u'earlier', u'damaged', u'lab', u'diagram', u'vocalization', u'jumping', u'refract', u'yellowfever', u'spectrometer', u'law', u'lap', u'types', u'goldhow', u'whale', u'chemicals', u'repeats', u'attempt', u'effective', u'move', u'recombination', u'antidiuretic', u'fungi', u'childhood', u'hypotheses', u'owls', u'complexity', u'maintain', u'green', u'ultimate', u'enter', u'fan', u'operate', u'athletes', u'order', u'wind', u'upon', u'restriction', u'helens', u'fetus', u'feedback', u'crosspollinates', u'fall', u'vary', u'sickle', u'parts', u'niko', u'crosspollinated', u'hibernation', u'gyres', u'london', u'affect', u'japan', u'budding', u'antacid', u'semipermeable', u'digest', u'wilting', u'surviving', u'auditory', u'personal', u'fix', u'blown', u'stomata', u'exhibits', u'writing', u'sprays', u'production', u'rhinoceroses', u'biotechnology', u'weeks', u'insecticide', u'easier', u'overcome', u'ball', u'eventually', u'affected', u'diseases', u'combination', u'prevented', u'mountainside', u'exhaling', u'safe', u'collide', u'break', u'band', u'bang', u'mutualistic', u'effects', u'highefficiency', u'schools', u'comprised', u'silver', u'group', u'structural', u'niches', u'represents', u'photosynthetic', u'alex', u'rocky', u'oxygen', u'brightness', u'grasslands', u'classified', u'hqt', u'l', u'rocks', u'wegner', u'loss', u'component', u'catabolic', u'reasonable', u'fitness', u'volcano', u'feeds', u'higher', u'classifies', u'side', u'bone', u'nonliving', u'interruption', u'seismometers', u'shuttle', u'us', u'snakes', u'evolve', u'series', u'platelets', u'cmb', u'carry', u'used', u'biomolecules', u'aids', u'flock', u'wiring', u'distantly', u'sounds', u'nio', u'large', u'reused', u'dating', u'gradient', u'paleontologist', u'tried', u'crucial', u'vessels', u'protostar', u'content', u'method', u'weighted', u'adapt', u'medicine', u'got', u'newly', u'size', u'sliding', u'centauri', u'foundation', u'splashed', u'perception', u'turning', u'barrier', u'colorless', u'given', u'free', u'flatworm', u'unconvinced', u'ancient', u'formation', u'rain', u'biological', u'sheet', u'detecting', u'wanted', u'storm', u'telescopes', u'chlorine', u'enormous', u'logging', u'recovering', u'created', u'starts', u'inexpensive', u'messages', u'days', u'tolerant', u'filter', u'disturbed', u'atp', u'soda', u'liquids', u'ribosome', u'onto', u'purpose', u'industrial', u'features', u'stopwatch', u'grade', u'carbonate', u'lowenergy', u'primary', u'hearing', u'differentation', u'wash', u'synthesize', u'floors', u'another', u'insulation', u'categorize', u'thick', u'plastic', u'electronic', u'illustrate', u'mercury', u'agriculture', u'toy', u'activities', u'similarly', u'geosphere', u'top', u'cooling', u'approximately', u'needed', u'rates', u'observations', u'burgess', u'phones', u'influence', u'percentage', u'john', u'bitter', u'dogs', u'damaging', u'glucose', u'ceiling', u'halite', u'intestine', u'collapse', u'mechanism', u'tool', u'leaking', u'took', u'wastes', u'western', u'helen', u'injected', u'subduction', u'begins', u'distance', u'intrusion', u'showed', u'hike', u'crosses', u'evaporates', u'second', u'seated', u'final', u'project', u'matter', u'pneumonic', u'flame', u'nervous', u'iron', u'peacocks', u'quarry', u'biochemical', u'enables', u'solely', u'supercontinent', u'acquisition', u'chirps', u'bridge', u'manned', u'sees', u'backflow', u'pollination', u'seep', u'ran', u'navigate', u'modern', u'mind', u'spectrum', u'talking', u'rat', u'seed', u'manner', u'seen', u'seem', u'regulate', u'tells', u'relatively', u'forced', u'strength', u'mostly', u'relayed', u'responsible', u'snow', u'recommended', u'causing', u'absorbed', u'forces', u'hatch', u'transmit', u'isolates', u'effectiveness', u'completed', u'grewwhich', u'even', u'weeding', u'wrists', u'though', u'iceland', u'object', u'shale', u'accurately', u'anatomy', u'reticulum', u'regular', u'mouth', u'raindrops', u'phase', u'pest', u'drought', u'watercress', u'replenishes', u'cellwhich', u'maria', u'crayfish', u'episode', u'treats', u'adenine', u'oath', u'medical', u'nighttime', u'dog', u'swamp', u'points', u'principle', u'sun', u'mating', u'calculates', u'incomplete', u'agnes', u'planetary', u'aquatic', u'hunger', u'insects', u'revolves', u'causes', u'probe', u'pathogens', u'attempts', u'radio', u'asia', u'lethal', u'earth', u'earthasteroid', u'nevada', u'availability', u'transport', u'departed', u'peck', u'spite', u'student', u'situations', u'explain', u'riverbed', u'nesting', u'sugar', u'identifies', u'bush', u'geologist', u'touched', u'rich', u'adequate', u'laptop', u'nebular', u'petri', u'wearing', u'plate', u'larvae', u'mixture', u'fossil', u'professionals', u'overflows', u'conduct', u'watch', u'perceive', u'coast', u'next', u'vaccinations', u'differentcolored', u'report', u'impractical', u'disadvantage', u'saturn', u'dr', u'becomes', u'societies', u'bat', u'contributes', u'die', u'countries', u'exchanged', u'fields', u'passing', u'bay', u'twice', u'microscope', u'release', u'clean', u'freshwater', u'explanations', u'steam', u'transforms', u'respond', u'ethical', u'endosperm', u'erroneous', u'decides', u'reference', u'observes', u'drugresistant', u'testing', u'niche', u'batteries', u'observed', u'depends', u'decided', u'radius', u'result', u'bee', u'hammer', u'composed', u'best', u'subject', u'brazil', u'oceanic', u'said', u'capacity', u'lots', u'away', u'rings', u'artificial', u'continent', u'web', u'future', u'toad', u'finger', u'disprove', u'simplest', u'conductive', u'drawn', u'caves', u'approach', u'discovery', u'claws', u'men', u'terms', u'extend', u'nature', u'smelled', u'weak', u'however', u'ww', u'southeast', u'eukaryotic', u'cut', u'wear', u'recent', u'clues', u'news', u'flowering', u'herd', u'roller', u'improve', u'isotopes', u'received', u'climate', u'accident', u'aristotles', u'presentations', u'cow', u'reported', u'resistant', u'country', u'extracts', u'conclusions', u'heating', u'participating', u'larger', u'protection', u'planned', u'receives', u'himalayan', u'distinction', u'contribution', u'magnetosphere', u'adapted', u'asked', u'lenses', u'eradicate', u'collecting', u'appeared', u'divisions', u'trunk', u'mutations', u'epidemic', u'height', u'nonrenewable', u'active', u'karen', u'chlorophyll', u'viruses', u'path', u'pregnancy', u'humming', u'testable', u'grazing', u'san', u'conference', u'pedaling', u'asks', u'basis', u'cockroaches', u'seashell', u'ropes', u'three', u'tiny', u'quickly', u'much', u'replicate', u'interest', u'basic', u'expected', u'flexible', u'basin', u'pancreas', u'tallest', u'containers', u'location', u'website', u'recessive', u'life', u'occupies', u'deeper', u'eastern', u'quantities', u'concerning', u'worker', u'bladder', u'hershey', u'convergence', u'lift', u'chase', u'child', u'scrubbers', u'comfortable', u'neutral', u'tortoise', u'applied', u'recyclable', u'inhibit', u'save', u'tank', u'adaptations', u'reddish', u'interprets', u'asthenosphere', u'stemcell', u'introduced', u'air', u'lesser', u'calcium', u'near', u'melted', u'suppose', u'teachers', u'leaves', u'applies', u'balance', u'flooding', u'anchor', u'procedure', u'mistake', u'cylinder', u'darwins', u'optical', u'mexico', u'equator', u'sanitizer', u'ii', u'soils', u'tissue', u'cone', u'elevation', u'mouse', u'disappear', u'grown', u'containing', u'specialization', u'cools', u'perform', u'suggest', u'make', u'goldfinches', u'wound', u'milliliters', u'linked', u'complex', u'potentially', u'phenotype', u'mixtures', u'vegetable', u'plates', u'colonies', u'several', u'grows', u'european', u'voyager', u'independent', u'satellite', u'peaks', u'theodore', u'disagreement', u'social', u'differing', u'buntings', u'hand', u'action', u'fuse', u'garlic', u'warden', u'earlobes', u'tuna', u'pacemakers', u'cycle', u'filtered', u'weakly', u'kept', u'scenario', u'changes', u'stimuli', u'failing', u'mainsequence', u'boxes', u'ocean', u'penzias', u'contact', u'greatest', u'prescribes', u'mother', u'claims', u'eagle', u'marsupials', u'cytosine', u'possesses', u'fittest', u'left', u'seasonal', u'wedging', u'molerats', u'proposed', u'sentence', u'sporting', u'distribute', u'rotated', u'gravity', u'assigned', u'proposes', u'identify', u'farther', u'human', u'facts', u'rotates', u'yet', u'previous', u'thyroid', u'adding', u'enters', u'monitored', u'hills', u'equipment', u'similarities', u'strains', u'milky', u'hay', u'belongs', u'transformed', u'cranberries', u'technological', u'concludes', u'allele', u'east', u'hat', u'gave', u'viking', u'atmospheres', u'usable', u'scrotum', u'ponds', u'advances', u'survival', u'thermometer', u'posed', u'possible', u'caps', u'possibly', u'fusion', u'rotate', u'birth', u'forested', u'cape', u'apart', u'burns', u'occurring', u'shoulder', u'melting', u'cooler', u'seaside', u'niels', u'divers', u'specific', u'pesticides', u'steps', u'mosquito', u'erodes', u'night', u'accuracy', u'fluoride', u'signal', u'cirrus', u'worldwide', u'beneficial', u'right', u'old', u'dimitri', u'deal', u'people', u'sends', u'deoxygenated', u'successfully', u'dead', u'born', u'meters', u'fourlegged', u'escape', u'altered', u'precautions', u'buoyancy', u'toxic', u'dense', u'undisturbed', u'bottom', u'electromagnetic', u'investigations', u'fox', u'okaloosae', u'daytime', u'polluted', u'contributing', u'fog', u'orbited', u'tertiary', u'nitrate', u'genotypes', u'cord', u'core', u'sherman', u'temperaturewhich', u'fish', u'defend', u'k', u'corn', u'burn', u'disappeared', u'shifting', u'defensive', u'graduated', u'eye', u'descendent', u'properties', u'shaken', u'refraction', u'carbonhydrogen', u'limitation', u'census', u'coral', u'months', u'ranges', u'moths', u'o', u'magazine', u'digestive', u'ensure', u'afternoon', u'steadily', u'islands', u'permission', u'octopus', u'match', u'nerve', u'discharging', u'digestion', u'statements', u'tetanus', u'presence', u'washed', u'float', u'obtaining', u'negative', u'device', u'beings', u'bats', u'coastal', u'balanced', u'thousands', u'reducing', u'tested', u'labrador', u'cones', u'existed', u'rely', u'cleared', u'compression', u'weathered', u'periodic', u'support', u'flying', u'knowing', u'transform', u'sunlight', u'fight', u'autosomes', u'quote', u'influencing', u'way', u'resulted', u'music', u'metabolism', u'launching', u'synthesis', u'lowest', u'intolerance', u'head', u'medium', u'gaseous', u'form', u'offer', u'forming', u'becoming', u'differences', u'removes', u'landing', u'back', u'failure', u'heat', u'needles', u'hear', u'solar', u'magellanic', u'removed', u'true', u'manatees', u'absent', u'responding', u'builds', u'reproductive', u'malaysia', u'prevents', u'inside', u'attached', u'maximum', u'devices', u'supplied', u'phosphate', u'cancers', u'lays', u'adenosine', u'unusually', u'panels', u'kite', u'accelerate', u'brightly', u'juvenile', u'medication', u'roamed', u'flat', u'megahertz', u'bacterium', u'ingest', u'muscular', u'convergent', u'gland', u'jupiter', u'saltwater', u'curious', u'sprouts', u'adversely', u'widows', u'mold', u'trip', u'fraternal', u'physical', u'circulatory', u'marking', u'relay', u'weakened', u'floor', u'glacier', u'na', u'whereas', u'noninterchangeable', u'generally', u'interested', u'role', u'holding', u'papers', u'test', u'erosion', u'company', u'unbalanced', u'ordovician', u'superposition', u'palms', u'delivered', u'models', u'felt', u'uniformly', u'diet', u'transported', u'class', u'fell', u'ridges', u'variable', u'transfers', u'carries', u'died', u'began', u'jones', u'younger', u'meteor', u'longer', u'megan', u'glass', u'assume', u'telescopic', u'grams', u'landed', u'together', u'freeze', u'beds', u'gorilla', u'rough', u'amniotic', u'time', u'push', u'serious', u'nuclei', u'paige', u'peacock', u'concept', u'cincinnati', u'chain', u'dance', u'fulcrum', u'activated', u'focus', u'leads', u'biotic', u'supplement', u'microwave', u'skin', u'dalton', u'abundant', u'displaying', u'milk', u'row', u'layers', u'asexual', u'inhabited', u'depend', u'disorders', u'dominated', u'flask', u'graph', u'technique', u'father', u'crater', u'environment', u'answered', u'charge', u'electron', u'supplements', u'rhythm', u'marks', u'discovering', u'hammers', u'saharan', u'ulna', u'division', u'protective', u'string', u'advantage', u'yeast', u'could', u'none', u'choice', u'automobiles', u'vegas', u'orientations', u'tanks', u'stays', u'join', u'exact', u'minute', u'cool', u'respiratory', u'clouds', u'nectar', u'level', u'altering', u'turns', u'sedimentary', u'brother', u'woodlands', u'standards', u'leave', u'item', u'settle', u'team', u'depository', u'lever', u'sidewalk', u'tortoises', u'shoe', u'prevent', u'dries', u'revolution', u'says', u'occurrence', u'findings', u'trend', u'gasoline', u'actinides', u'discover', u'sign', u'lengthened', u'slowed', u'prairie', u'bombardment', u'worn', u'patient', u'bacterial', u'millionyearold', u'fear', u'spectral', u'endangering', u'stressful', u'appear', u'performs', u'axes', u'melt', u'uniform', u'current', u'energy', u'exertion', u'boobys', u'goes', u'supplies', u'falling', u'curled', u'filled', u'tetrapod', u'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00', u'supporting', u'predicting', u'explosion', u'dwarf', u'alkaline', u'genes', u'french', u'understanding', u'water', u'meteorites', u'\x00\x00\x00`\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\ufffd\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x02\x00\x00\x00\x00\x01\x00\x00\x04\x00\x00\x00\x00\x02\x00\x00\x08\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00', u'baseball', u'pasteurization', u'yards', u'kilometers', u'goldfish', u'along', u'constructive', u'teacher', u'change', u'knowledge', u'box', u'passengers', u'unidentified', u'incoming', u'canadian', u'healthy', u'shift', u'studied', u'funded', u'mendeleev', u'commonly', u'somatic', u'spheres', u'accomplished', u'franklin', u'usually', u'diploid', u'noticeably', u'studies', u'\x00\x00\x00\x00\x00\x00\x01@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', u'bog', u'tasks']

In [19]:
#print keywordlist[1400:2800]
keywordlist2 = [u'useful', u'hiking', u'extra', u'soybeans', u'storms', u'prefer', u'compare', u'marked', u'modifying', u'kayak', u'yellowcoated', u'controls', u'sufferers', u'hibernate', u'rarely', u'drug', u'sky', u'baby', u'bulbs', u'undergone', u'towards', u'nitrates', u'working', u'paleozoic', u'formulated', u'positive', u'tubes', u'tightly', u'emits', u'territories', u'live', u'going', u'prey', u'opposed', u'peru', u'criteria', u'encode', u'actin', u'australian', u'compressive', u'selects', u'household', u'today', u'helix', u'rainfall', u'conductor', u'ionic', u'club', u'organized', u'acceptable', u'apparent', u'validity', u'minnows', u'antigens', u'visual', u'rigid', u'microtubules', u'sulfur', u'cases', u'effort', u'countertops', u'fly', u'organizations', u'rotational', u'flagellum', u'lifethreatening', u'unbiased', u'car', u'originally', u'rutherford', u'flu', u'orion', u'longitude', u'cat', u'districts', u'soup', u'labeled', u'gathers', u'believed', u'making', u'modifies', u'arrive', u'pulls', u'placements', u'streak', u'heart', u'classrooms', u'grassy', u'stream', u'frogs', u'expanding', u'confused', u'agent', u'hatches', u'sample', u'heard', u'slows', u'dropped', u'doublemembrane', u'allowed', u'javiers', u'clothing', u'occur', u'hatchling', u'pink', u'lactase', u'rays', u'monitoring', u'anabolic', u'inheritance', u'cans', u'tilt', u'divided', u'spreads', u'sandstone', u'sized', u'pine', u'snapdragon', u'chemical', u'immune', u'hydrologist', u'botanist', u'fourth', u'elephant', u'repellent', u'eight', u'dominance', u'economy', u'map', u'product', u'ductile', u'laundry', u'mat', u'information', u'may', u'freezing', u'paleontologists', u'southern', u'needs', u'deforestation', u'produce', u'newer', u'designed', u'cardiac', u'drastic', u'objects', u'destructive', u'data', u'grow', u'lifting', u'man', u'classroom', u'abiotic', u'stress', u'natural', u'varieties', u'liquid', u'conscious', u'whenever', u'slightly', u'corona', u'st', u'metalloid', u'accelerated', u'truck', u'deposit', u'african', u'representation', u'wolves', u'increase', u'pulled', u'auxin', u'typical', u'wavelengths', u'presentation', u'acts', u'shoes', u'archeologist', u'repair', u'serve', u'gradual', u'pointed', u'beak', u'displayed', u'wildfires', u'years', u'aiding', u'course', u'episodes', u'experiments', u'nitrogen', u'cuts', u'marshmallows', u'cold', u'still', u'pointer', u'birds', u'blocked', u'machine', u'limbs', u'felicia', u'procedures', u'reacting', u'fullsize', u'vicinity', u'coordinates', u'forms', u'attraction', u'window', u'continue', u'farmer', u'\x00\x00\x00e\x00\x00\x01\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x04dsdb\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00', u'main', u'decades', u'visiting', u'texas', u'happened', u'chromatin', u'carpal', u'thereby', u'matches', u'conversations', u'phytoplankton', u'evolution', u'sooner', u'underneath', u'records', u'adrenaline', u'markings', u'base', u'half', u'feature', u'bath', u'killed', u'maintaining', u'highlevel', u'susceptibility', u'term', u'name', u'stabilize', u'january', u'subducts', u'belts', u'proteins', u'appropriate', u'legume', u'migrate', u'offshore', u'rock', u'veins', u'entirely', u'repaired', u'seldom', u'format', u'turtle', u'el', u'bicycle', u'nematode', u'significantly', u'elephants', u'ejaculation', u'sugars', u'abnormal', u'katia', u'conventions', u'wrap', u'fires', u'gaps', u'entering', u'sexually', u'wavelength', u'happen', u'adequately', u'plans', u'canada', u'living', u'ultimately', u'accomplish', u'container', u'opened', u'colbys', u'space', u'greeks', u'grasses', u'factory', u'thirst', u'looking', u'attracted', u'contained', u'exploding', u'emerged', u'dioxide', u'midatlantic', u'instructions', u'receiving', u'desertlike', u'correct', u'shows', u'clover', u'performing', u'theory', u'booby', u'defenses', u'cars', u'million', u'divergence', u'isotope', u'abdomen', u'binary', u'california', u'besides', u'himalaya', u'stone', u'keith', u'explorers', u'care', u'waxy', u'training', u'jakes', u'mutates', u'dinosaurs', u'estella', u'collapsing', u'transition', u'pandemic', u'kingdoms', u'motion', u'thing', u'butterflies', u'place', u'massive', u'swing', u'accelerating', u'shoemakerlevy', u'think', u'substrates', u'lay', u'origin', u'hydrocarbons', u'henry', u'whitetailed', u'symmetry', u'striking', u'variables', u'coming', u'nutritional', u'nested', u'handled', u'specifically', u'directly', u'invasion', u'comprises', u'impossible', u'ring', u'open', u'george', u'engineer', u'city', u'little', u'necessarily', u'checked', u'genotype', u'occupy', u'mendels', u'caught', u'structured', u'hypothesiscrickets', u'tusks', u'anyone', u'indicate', u'wilts', u'conservation', u'moisture', u'typing', u'white', u'returns', u'structures', u'national', u'williams', u'gives', u'bear', u'exploring', u'eyes', u'releases', u'season', u'hardyweinberg', u'continuous', u'eliminates', u'transverse', u'released', u'nutrientrich', u'stratospheric', u'absorbs', u'flowers', u'representing', u'logic', u'population', u'wide', u'television', u'cracks', u'iceberg', u'overtaken', u'accordance', u'endoplasmic', u'favors', u'require', u'crowded', u'transferred', u'war', u'coats', u'protected', u'magnesium', u'fruits', u'sweeteners', u'outcome', u'photosynthesis', u'scuba', u'decompose', u'inferring', u'pond', u'addressing', u'illness', u'mated', u'wooded', u'remained', u'craig', u'topics', u'grounddwelling', u'turned', u'locations', u'generalization', u'bugs', u'stratosphere', u'rained', u'microwavesafe', u'made', u'stanley', u'exchanging', u'sells', u'saw', u'approaching', u'marathon', u'sat', u'rolanda', u'conversion', u'exploratory', u'greek', u'efficient', u'answering', u'ideas', u'hotspot', u'produces', u'longshore', u'endangered', u'overlapping', u'potential', u'take', u'repeatedly', u'higheryou', u'interior', u'destroy', u'glassware', u'biscayne', u'heal', u'channel', u'butterfly', u'formulas', u'begin', u'sure', u'multiple', u'midpacific', u'opposite', u'track', u'knew', u'microwaves', u'suspects', u'evaporate', u'falls', u'boiling', u'wire', u'rationale', u'pair', u'america', u'interacting', u'planning', u'michelle', u'lawn', u'vapor', u'radioactive', u'renal', u'enamel', u'protists', u'considered', u'average', u'later', u'infections', u'\x00\x00\x00\x00\x00\x01\x00@\x00\x00\x00\x00\x00\x01\x00\ufffd\x00\x00\x00\x00\x00\x01\x01\x00\x00\x00\x00\x00\x00\x01\x02\x00\x00\x00\x00\x00\x00\x01\x04\x00\x00\x00\x00\x00\x00\x01\x08\x00\x00\x00\x00\x00\x00\x01\x10\x00\x00\x00\x00\x00\x00\x01', u'tooth', u'sunset', u'mri', u'predatory', u'clearing', u'grassland', u'atlantic', u'dispersed', u'duplication', u'typically', u'quantity', u'slope', u'permeable', u'nebula', u'heap', u'regulated', u'show', u'german', u'circulating', u'minutes', u'darkcolored', u'friction', u'cells', u'discovered', u'bright', u'spaces', u'helena', u'inconsistent', u'uranus', u'encoded', u'iridiumcontaining', u'aggressive', u'label', u'line', u'fertilization', u'ground', u'boundaries', u'upgrade', u'ratio', u'gulf', u'trigger', u'plume', u'stained', u'proportion', u'enough', u'texture', u'lightcolored', u'wood', u'black', u'lipids', u'developed', u'sierra', u'depletion', u'stain', u'caroline', u'extracellular', u'carolina', u'lighted', u'narrows', u'correctly', u'get', u'contracts', u'revolutionized', u'moons', u'designated', u'pepsin', u'mission', u'reproducible', u'reptile', u'cannot', u'nearly', u'primates', u'secondary', u'regarding', u'resource', u'skull', u'aluminum', u'settings', u'floods', u'hinges', u'saliva', u'yield', u'morning', u'miles', u'roger', u'predicted', u'bobcats', u'sonar', u'pituitary', u'discussing', u'communicated', u'vision', u'reveals', u'burst', u'subatomic', u'spore', u'soot', u'isabel', u'actively', u'college', u'horned', u'collects', u'insoluble', u'diver', u'concern', u'surfaces', u'herschel', u'detect', u'voluntary', u'compounds', u'ways', u'subsequent', u'review', u'definite', u'sites', u'forecast', u'vaccines', u'outside', u'roots', u'radiated', u'diamonds', u'herbal', u'import', u'reading', u'across', u'notice', u'straightening', u'parent', u'era', u'originated', u'scavengers', u'screen', u'flow', u'killing', u'sandstorms', u'toothpaste', u'jets', u'article', u'spark', u'cities', u'come', u'mountaintops', u'isthmus', u'tug', u'influenza', u'reaction', u'circumference', u'javier', u'efficiency', u'many', u'region', u'according', u'contract', u'peptidoglycan', u'rigel', u'tour', u'dated', u'comet', u'expression', u'comes', u'nearby', u'yearly', u'among', u'filtering', u'cancer', u'color', u'pot', u'colony', u'period', u'pop', u'sampling', u'threaten', u'analyzed', u'pole', u'learning', u'geese', u'extinct', u'ptolemys', u'consider', u'runaway', u'sticks', u'caring', u'better', u'combines', u'cycling', u'consumptive', u'dumping', u'west', u'rebuild', u'locally', u'vacation', u'relatedness', u'cared', u'peruvian', u'collided', u'deposited', u'combined', u'mars', u'helicopter', u'rebuilt', u'reflex', u'earths', u'differently', u'wants', u'lactose', u'direction', u'enable', u'pesticide', u'thousand', u'formed', u'neutrinos', u'wires', u'readings', u'observers', u'geothermal', u'observe', u'oils', u'external', u'foggy', u'evidence', u'squirrels', u'actions', u'sound', u'soccer', u'turtles', u'developing', u'might', u'mount', u'single', u'w', u'diverse', u'ongoing', u'familiar', u'promising', u'squeezed', u'investigating', u'converted', u'taking', u'polymer', u'adds', u'canoe', u'tens', u'characteristics', u'rubbing', u'habitats', u'enlarges', u'equal', u'middle', u'bias', u'converts', u'eras', u'\x00\x00\x00\x01\x00\x00\x00@\x00\x00\x00\x01\x00\x00\x00\ufffd\x00\x00\x00\x01\x00\x00\x01\x00\x00\x00\x00\x01\x00\x00\x02\x00\x00\x00\x00\x01\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x10\x00\x00\x00\x00\x01\x00\x00', u'protein', u'property', u'technology', u'cytoskeleton', u'tourist', u'bags', u'different', u'revolutions', u'develop', u'beakers', u'doctor', u'seats', u'theorized', u'granted', u'scientifically', u'woodland', u'lava', u'check', u'welladapted', u'inquiry', u'buys', u'lifetimes', u'streams', u'constructed', u'events', u'pan', u'workplace', u'week', u'exhaust', u'extended', u'barefoot', u'vestigial', u'fossilfuel', u'rainbow', u'prominences', u'foam', u'drives', u'nephron', u'producers', u'persons', u'altitude', u'fruit', u'investigates', u'changing', u'validate', u'oral', u'canopy', u'renewable', u'treeshaded', u'relies', u'antibodies', u'nevadas', u'traps', u'volcanoes', u'charges', u'sinkhole', u'severe', u'without', u'engulfing', u'subsystems', u'relied', u'coordinate', u'enceladus', u'model', u'researchers', u'charged', u'summer', u'sprayed', u'noah', u'transmitting', u'compound', u'neon', u'wellrounded', u'rest', u'communities', u'lipid', u'kill', u'aspect', u'differentiation', u'speed', u'captured', u'insight', u'rover', u'lightbulbs', u'death', u'generator', u'womans', u'rose', u'photons', u'conclude', u'seems', u'except', u'geological', u'meteorite', u'bioenergy', u'formative', u'treatment', u'samples', u'hind', u'pili', u'electrical', u'extensive', u'heavier', u'genome', u'kingdom', u'jake', u'aspects', u'gastroenterologist', u'kills', u'read', u'outermost', u'overall', u'decomposed', u'\x00\x00\x00\x00\x01\x00\x00@\x00\x00\x00\x00\x01\x00\x00\ufffd\x00\x00\x00\x00\x01\x00\x01\x00\x00\x00\x00\x00\x01\x00\x02\x00\x00\x00\x00\x00\x01\x00\x04\x00\x00\x00\x00\x00\x01\x00\x08\x00\x00\x00\x00\x00\x01\x00\x10\x00\x00\x00\x00\x00\x01\x00', u'early', u'heterozygous', u'traffic', u'ants', u'listening', u'germinatewhich', u'pangea', u'world', u'accepted', u'amu', u'decomposer', u'glaciers', u'bonding', u'fossils', u'particle', u'pounds', u'epicycles', u'smell', u'reduces', u'conducts', u'pharm', u'maintained', u'serves', u'lightning', u'discarded', u'benefit', u'either', u'clay', u'served', u'downward', u'stationary', u'reduced', u'bench', u'sharp', u'twelve', u'exposed', u'sequences', u'reversed', u'catalytic', u'competition', u'torricelli', u'nucleic', u'magma', u'images', u'intact', u'organizing', u'thinks', u'hairline', u'downthrown', u'viral', u'caribbean', u'calculating', u'symmetricallyshaped', u'tube', u'stages', u'recorded', u'stops', u'ddt', u'pioneer', u'exit', u'lunar', u'expressing', u'amino', u'paleoclimatologists', u'decomposers', u'measuring', u'considering', u'biased', u'backbone', u'shadow', u'scientific', u'power', u'epsom', u'seconds', u'retains', u'seedlings', u'communicate', u'unusual', u'sandpaper', u'broken', u'fertilized', u'ratios', u'refers', u'went', u'legumes', u'manufacturer', u'pressurize', u'comparison', u'conducting', u'central', u'ferromagnetic', u'package', u'island', u'paralysis', u'discussed', u'mrna', u'mixes', u'practical', u'mutually', u'hypothalamus', u'wolf', u'greatly', u'act', u'mixed', u'lasting', u'road', u'checkup', u'fullsized', u'researching', u'urey', u'fertile', u'decreased', u'marshmallow', u'weighs', u'press', u'communication', u'image', u'observatory', u'charts', u'spinal', u'newtons', u'accounts', u'determine', u'heated', u'accumulated', u'feeders', u'telescope', u'arctic', u'tropical', u'fast', u'prepare', u'area', u'whales', u'removing', u'phobos', u'hazards', u'disposed', u'desert', u'start', u'appears', u'cats', u'low', u'stars', u'secretion', u'cultures', u'hen', u'valley', u'materials', u'treat', u'billion', u'complete', u'secreted', u'mammals', u'strikes', u'secretes', u'posterior', u'land', u'ended', u'tall', u'hurricanes', u'reflective', u'mice', u'trying', u'hottest', u'faster', u'pull', u'waste', u'arranged', u'largest', u'circulation', u'animal', u'appliance', u'turbines', u'wet', u'strawberry', u'snowshoes', u'cabin', u'emissions', u'tones', u'navel', u'diagrams', u'grass', u'builders', u'agree', u'rust', u'expect', u'detailed', u'gone', u'carver', u'tectonic', u'whiteflowered', u'certain', u'describe', u'deer', u'deep', u'general', u'examine', u'lichen', u'au', u'daily', u'planets', u'laboratory', u'salvage', u'moves', u'spends', u'film', u'ice', u'statementdespite', u'beyond', u'raised', u'allergy', u'rubbed', u'credibility', u'storage', u'event', u'bloodsucking', u'field', u'prism', u'valid', u'astronaut', u'weights', u'residence', u'pollutants', u'coughing', u'poor', u'polar', u'releasing', u'vaccinate', u'metal', u'separate', u'students', u'igneous', u'teeth', u'drift', u'carpenter', u'includes', u'important', u'flattened', u'peak', u'transporting', u'brands', u'revolve', u'sneeze', u'pool', u'building', u'tiger', u'farms', u'condensation', u'individual', u'controlled', u'reactive', u'mitochondria', u'organelles', u'kilogram', u'reproduce', u'dramatic', u'mass', u'lightbulb', u'behavioral', u'nutrient', u'ovary', u'accreted', u'starting', u'herbivores', u'levers', u'externally', u'emile', u'skeleton', u'chinese', u'urinary', u'suns', u'asteroids', u'unique', u'splitting', u'seals', u'sunk', u'spacecraft', u'debris', u'reevaluated', u'zinc', u'tectonics', u'dish', u'follow', u'covalent', u'disk', u'mining', u'children', u'oaths', u'emily', u'particles', u'careful', u'referring', u'hunting', u'tv', u'startled', u'subjected', u'sulfate', u'removal', u'autosomal', u'program', u'ecosystem', u'preserved', u'th', u'safety', u'centromere', u'twothirds', u'urea', u'sprains', u'returning', u'far', u'rod', u'floated', u'subunits', u'oxides', u'ultraviolet', u'reordering', u'decide', u'bleach', u'atmospheric', u'bubbles', u'difference', u'capillary', u'appropriately', u'exotic', u'splits', u'treating', u'nahco', u'washing', u'mesosaurus', u'similarsized', u'skeletal', u'industrialization', u'rocket', u'laying', u'prolonged', u'zygote', u'grandfather', u'trench', u'dinosaur', u'sand', u'highway', u'inversions', u'mammal', u'smith', u'sank', u'physically', u'antibiotic', u'vertical', u'ten', u'jenkinss', u'lecture', u'lack', u'healed', u'translating', u'go', u'rate', u'invention', u'design', u'pass', u'addition', u'childs', u'consumers', u'disruptions', u'section', u'lawrence', u'crust', u'ammonia', u'run', u'scientists', u'negativelycharged', u'learned', u'hooke', u'public', u'contrast', u'movement', u'condensed', u'full', u'recruits', u'malaria', u'litters', u'electricgasolinehybrid', u'owns', u'modify', u'answers', u'vomiting', u'hours', u'cylindrical', u'operating', u'compressed', u'excess', u'directions', u'rolando', u'buzzing', u'strong', u'observing', u'hormones', u'labs', u'chipped', u'colored', u'panthers', u'stretching', u'allows', u'jackie', u'experience', u'round', u'prior', u'amount', u'healthier', u'breakdown', u'published', u'runoff', u'homologous', u'skepticism', u'specialize', u'elongated', u'cystic', u'paired', u'biodegradable', u'via', u'followed', u'family', u'suddenly', u'requiring', u'africa', u'layer', u'sneezing', u'sinkholes', u'vertebral', u'hogs', u'asexually', u'analyzing', u'appliances', u'inorganic', u'extinction', u'establish', u'estimate', u'earliest', u'flaws', u'takes', u'distinct', u'contains', u'petroleum', u'two', u'generate', u'comparing', u'unrecognizable', u'soil', u'taken', u'ovules', u'injury', u'wiper', u'wipes', u'phone', u'horses', u'ladybug', u'\x00\x00\x00\x01bud1\x00\x00\x10\x00\x00\x00\x08\x00\x00\x00\x10\x00\x00\x00\x01\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x10\x00\x00n\x00_\x00q\x00u\x00e\x00s\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x002\x00t\x00r\x00a\x00i\x00n\x00_\x00q\x00u\x00e\x00s\x00t\x00i\x00o\x00n\x00.\x00t\x00x\x00t\x00_\x00t\x00o\x00_\x00u\x00n\x00i\x00c\x00o\x00d\x00e\x00_\x00r\x00e\x00m\x00o\x00v\x00e\x00_\x00s\x00t\x00o\x00p\x00w\x00o\x00r\x00d\x00s\x00.\x00t\x00x\x00tilocblob\x00\x00\x00\x10\x00\x00\x00[\x00\x00\x000\x00\x00\x00\x10\ufffd\ufffd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x08', u'lander', u'knows', u'initiated', u'substances', u'inhabits', u'propellers', u'hydrological', u'corrected', u'emission', u'galileos', u'itjack', u'initiates', u'flag', u'surgically', u'controlling', u'impulse', u'particular', u'known', u'coyote', u'suction', u'factories', u'producing', u'analogous', u'weathering', u'largescale', u'town', u'helped', u'keeping', u'nonvolcanic', u'hour', u'science', u'supergiant', u'yaxis', u'protons', u'generated', u'friendly', u'taus', u'remain', u'sudden', u'indicating', u'evolved', u'learn', u'knocked', u'sahara', u'disposal', u'male', u'specialized', u'marble', u'history', u'unknown', u'ions', u'shark', u'prompt', u'pushes', u'stated', u'aphids', u'joints', u'trial', u'share', u'accept', u'm', u'states', u'collision', u'methyl', u'protects', u'numbers', u'hardcrystalline', u'sense', u'cows', u'phrase', u'lacks', u'species', u'bonded', u'axis', u'transduction', u'huge', u'interbreed', u'unexpected', u'goal', u'pinnacles', u'rather']


In [20]:
#print keywordlist[2800:4200]
keywordlist3 = [u'helium', u'breaking', u'utilize', u'stayed', u'everyday', u'maps', u'glowing', u'occasionally', u'spotted', u'influences', u'ingesting', u'centimeters', u'movements', u'explains', u'plant', u'sandwich', u'calvaria', u'communicating', u'contribute', u'rainforest', u'derived', u'orbits', u'seaturtle', u'fungus', u'tries', u'plane', u'lighting', u'malnourished', u'softball', u'mendel', u'horizontal', u'reclassify', u'bloom', u'response', u'dissolving', u'short', u'cellstudent', u'resemble', u'enzymes', u'coat', u'twisting', u'eats', u'meat', u'coal', u'shore', u'broadcasting', u'responsibility', u'shade', u'maneuver', u'geographic', u'egg', u'equation', u'grasshoppers', u'rounded', u'help', u'reservoir', u'sooj', u'urine', u'soon', u'installed', u'held', u'thermal', u'organize', u'destruction', u'signs', u'southwestern', u'existence', u'kinetic', u'suffer', u'environmentally', u'cardiovascular', u'reactant', u'jacobs', u'arteries', u'rapidly', u'slipped', u'pests', u'sodium', u'energies', u'houseplant', u'insecteating', u'handheld', u'actually', u'late', u'resort', u'detected', u'wildcats', u'microscopic', u'systems', u'symptoms', u'pad', u'tugofwar', u'sauce', u'alter', u'stability', u'good', u'return', u'calm', u'impacts', u'deforested', u'food', u'propose', u'males', u'predator', u'floating', u'rainforests', u'estuaries', u'walls', u'saffirsimpson', u'weeds', u'foot', u'cumulonimbus', u'nucleus', u'landscapers', u'viewers', u'cambrian', u'develops', u'easily', u'bleeds', u'gets', u'increasewhich', u'mathematician', u'characterizes', u'presented', u'evade', u'someone', u'stopped', u'predators', u'maltose', u'radial', u'found', u'repeating', u'characterized', u'referred', u'heavy', u'reactions', u'harm', u'england', u'bends', u'weight', u'generation', u'colder', u'hare', u'hard', u'reduce', u'idea', u'hawaii', u'engaging', u'investigated', u'connect', u'measurement', u'windstorm', u'cemented', u'mates', u'orient', u'assigns', u'travels', u'flower', u'divide', u'alcohol', u'navigational', u'robert', u'surrounded', u'projected', u'publish', u'research', u'si', u'acting', u'sustain', u'hill', u'schleiden', u'dying', u'motorcycle', u'occurs', u'gardener', u'colleges', u'drink', u'jeremiah', u'risen', u'drifting', u'favored', u'timeline', u'seismogram', u'dung', u'dune', u'effect', u'pleasant', u'houses', u'reason', u'basketball', u'algal', u'brightest', u'members', u'put', u'equipped', u'ash', u'beginning', u'algae', u'thrown', u'definition', u'pairs', u'tsunamis', u'launch', u'brown', u'metamorphosis', u'computers', u'positioning', u'finches', u'accumulation', u'whitetail', u'mantis', u'conducted', u'edges', u'sensations', u'circuit', u'bumper', u'consequence', u'undergo', u'fallen', u'throws', u'reinforced', u'feed', u'copper', u'major', u'mr', u'drooling', u'dont', u'probability', u'upper', u'feel', u'triplebeam', u'jogs', u'number', u'nacl', u'traveled', u'instances', u'utilized', u'drier', u'done', u'least', u'soapy', u'mist', u'alligators', u'earthworms', u'meadow', u'herbicides', u'passes', u'differ', u'landmass', u'temperature', u'jet', u'introduction', u'leading', u'interact', u'coasts', u'construct', u'regulation', u'station', u'statement', u'obsoleta', u'compromise', u'majority', u'nuts', u'lichens', u'banana', u'store', u'kailey', u'shifted', u'repetitive', u'feels', u'passed', u'glow', u'behind', u'month', u'forests', u'jamie', u'park', u'antibacterial', u'required', u'everglades', u'assumption', u'calf', u'signaling', u'doctors', u'believe', u'\x00\x00\x00\x00\x00\x00\x01@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x10', u'brackish', u'center', u'linking', u'jump', u'kind', u'farming', u'determines', u'polysaccharides', u'double', u'largely', u'grey', u'recording', u'selfdigestion', u'autumn', u'determined', u'contrary', u'ecuador', u'elaborate', u'interacts', u'gps', u'playing', u'electrically', u'enclosed', u'sunblock', u'exert', u'strengthen', u'stimulates', u'ages', u'ninetynine', u'randomly', u'juice', u'antarctica', u'muscles', u'lawnmower', u'depleted', u'categorized', u'seafloor', u'carrying', u'concentration', u'bohr', u'organs', u'orders', u'vulnerable', u'mountain', u'wilson', u'nights', u'relationship', u'zero', u'digests', u'pain', u'cave', u'proportional', u'trash', u'camouflage', u'bacteria', u'lit', u'moist', u'internal', u'square', u'build', u'analogy', u'finding', u'gym', u'rna', u'gill', u'frequently', u'play', u'added', u'hires', u'bottle', u'index', u'mendozas', u'retinal', u'eggs', u'measures', u'eruption', u'chart', u'react', u'inability', u'virus', u'nasas', u'experiences', u'immediate', u'significant', u'buried', u'alpha', u'measured', u'flares', u'stands', u'reestablishes', u'joanne', u'kg', u'mineral', u'windows', u'clear', u'sometimes', u'coincides', u'salt', u'km', u'salinity', u'concepts', u'dissolved', u'part', u'electrons', u'bodies', u'nonspecific', u'narrow', u'velocity', u'ledgeswhich', u'notch', u'organism', u'cooled', u'lying', u'favorable', u'stones', u'reefs', u'golf', u'phenomenon', u'gold', u'nonmetals', u'heliumfilled', u'scorched', u'evaporation', u'hillside', u'inflexible', u'outdoor', u'relation', u'umbilicus', u'repels', u'stores', u'breezes', u'find', u'faces', u'impact', u'cell', u'indicator', u'unusable', u'deserts', u'northern', u'grazed', u'discshaped', u'condor', u'distributed', u'incidence', u'triphosphate', u'beat', u'enzyme', u'experiment', u'colby', u'symptom', u'eruptions', u'factor', u'poles', u'circle', u'darwin', u'oil', u'represent', u'porcupine', u'diffusion', u'lithospheric', u'hit', u'therefore', u'hiv', u'biogeochemical', u'dependent', u'mower', u'william', u'alicia', u'sunny', u'dry', u'trees', u'obtain', u'mutualism', u'whistle', u'capsule', u'grew', u'fastest', u'closely', u'compass', u'combinations', u'testes', u'vein', u'running', u'molten', u'shines', u'californias', u'resolve', u'roles', u'recycled', u'paddling', u'coasted', u'remove', u'garcia', u'investigate', u'stored', u'ballsocket', u'doppler', u'activity', u'x', u'creates', u'river', u'airfilled', u'wrote', u'locating', u'satisfy', u'bars', u'set', u'achieved', u'enabled', u'vertebrates', u'advertised', u'vine', u'sex', u'touch', u'see', u'arc', u'bare', u'migration', u'sea', u'chambers', u'precursors', u'geologic', u'constellation', u'arm', u'declined', u'barn', u'preparation', u'learns', u'mechanically', u'probable', u'pneumonia', u'aqueduct', u'pictures', u'currently', u'burner', u'fission', u'degrading', u'smallest', u'various', u'homeostasis', u'vascular', u'orbiting', u'probably', u'burned', u'conditions', u'europe', u'carbonated', u'makeup', u'diseasecausing', u'creating', u'coalfired', u'missing', u'keyboard', u'lunch', u'craters', u'following', u'spray', u'saturns', u'selectively', u'chemiosmotic', u'litter', u'invisible', u'vesicles', u'delivering', u'remaining', u'floats', u'strontium', u'c', u'toward', u'improved', u'palomar', u'components', u'density', u'joint', u'phosphorous', u'sensitive', u'improves', u'residential', u'collection', u'nitrogenrich', u'mean', u'whole', u'finds', u'experimental', u'millions', u'flies', u'regulates', u'groups', u'canadas', u'supply', u'simple', u'sentenceatmospheric', u'acted', u'address', u'community', u'deposits', u'dimensional', u'galileo', u'unicellular', u'simply', u'adaptation', u'luis', u'vessel', u'mattress', u'esophagus', u'throughout', u'goats', u'belt', u'decline', u'consuming', u'implanted', u'described', u'raise', u'symposium', u'instrument', u'monthly', u'unaided', u'meteorologists', u'membranes', u'due', u'fizzing', u'earthquake', u'hitting', u'runner', u'reduction', u'dropping', u'galilei', u'las', u'collected', u'pile', u'ph', u'territory', u'meeting', u'empty', u'terrence', u'generating', u'lived', u'flight', u'advertise', u'boy', u'fire', u'firefighters', u'crack', u'gas', u'precision', u'gap', u'engulfed', u'moderation', u'biodiversity', u'understand', u'liver', u'pacemaker', u'demand', u'prices', u'instructor', u'struck', u'hardness', u'plants', u'extracted', u'bile', u'georgia', u'classifications', u'look', u'grains', u'frozen', u'straight', u'rope', u'replaced', u'adh', u'evaluating', u'impacted', u'sheets', u'nucleotides', u'biking', u'loop', u'engineers', u'real', u'underground', u'approximate', u'century', u'spontaneous', u'coalfueled', u'sentences', u'suspended', u'geologists', u'alaska', u'destabilization', u'reads', u'wildflowers', u'ready', u'rid', u'periodically', u'alone', u'irrigate', u'currents', u'around', u'javelinas', u'taxonomy', u'chloroplasts', u'shirt', u'glycogen', u'seismographs', u'belong', u'galapagos', u'interferes', u'elevated', u'shorter', u'rules', u'arabidopsis', u'researcher', u'smallpox', u'widely', u'grand', u'products', u'nekton', u'ray', u'locationsone', u'composition', u'resources', u'diagnosed', u'lystrosaurus', u'strips', u'temporary', u'snail', u'lie', u'fightorflight', u'alert', u'viewing', u'levels', u'moving', u'cleaning', u'louis', u'bees', u'plugs', u'nonmetal', u'focal', u'stoplights', u'laws', u'dark', u'lower', u'motionless', u'older', u'accumulate', u'makes', u'redesigned', u'analysis', u'picks', u'sports', u'person', u'boat', u'edge', u'regulating', u'y', u'organization', u'bounced', u'extinctions', u'pollen', u'things', u'lobsters', u'eyeglasses', u'hypothesized', u'beetle', u'named', u'competitive', u'isolated', u'abbreviation', u'professors', u'shape', u'intervals', u'atomic', u'questions', u'using', u'inference', u'alternative', u'praying', u'fur', u'schwanns', u'letting', u'onset', u'loading', u'preparations', u'tablet', u'cup', u'bodys', u'decomposes', u'danger', u'reactivity', u'signals', u'source', u'subjects', u'parents', u'kites', u'quick', u'input', u'transformation', u'farmland', u'australia', u'homozygous', u'rodents', u'fusions', u'demands', u'big', u'showing', u'doorbell', u'doses', u'vital', u'manatee', u'competing', u'undercurrents', u'game', u'marcy', u'collectively', u'ecosystems', u'individually', u'precipitation', u'wings', u'helpful', u'insect', u'skeptical', u'modifications', u'sorting', u'lost', u'shaped', u'polymers', u'lacking', u'disks', u'chest', u'productive', u'hemisphere', u'shapes', u'collect', u'reasons', u'individuals', u'indication', u'byproduct', u'nutrients', u'translated', u'fleas', u'explainable', u'biospherein', u'disorder', u'refracted', u'mathematical', u'ecology', u'often', u'celled', u'metabolic', u'passage', u'spring', u'fathers', u'creation', u'differentiate', u'bounce', u'global', u'traits', u'strongest', u'pack', u'examples', u'erupts', u'candle', u'stir', u'preparing', u'electric', u'ozone', u'gradually', u'scale', u'describing', u'jewelry', u'twinkle', u'closest', u'affects', u'decision', u'measurements', u'firecracker', u'audience', u'per', u'raising', u'alfred', u'hydroxide', u'eliminate', u'tends', u'fragments', u'thirdclass', u'mitotic', u'pea', u'exercising', u'varying', u'eating', u'reach', u'jumps', u'lose', u'cushioning', u'smiths', u'stem', u'step', u'wells', u'lasts', u'become', u'plots', u'watered', u'raspberry', u'intense', u'vaccine', u'wealth', u'looped', u'lowercost', u'wildlife', u'cautious', u'extending', u'completing', u'puddles', u'heterotrophs', u'range', u'involving', u'beans', u'wonders', u'shiny', u'farmers', u'gamma', u'pollution', u'fern', u'seeing', u'eaten', u'garbage', u'undifferentiated', u'wegener', u'within', u'invading', u'transformations', u'onshore', u'nothing', u'stalactites', u'mitosis', u'mitochondrial', u'cretaceous', u'primarily', u'mangrove', u'contributor', u'drifted', u'protoplanets', u'rolls', u'smells', u'arrow', u'poaching', u'computer', u'practices', u'ptolemy', u'extremely', u'placing', u'question', u'duplicate', u'gases', u'moseley', u'asteroid', u'infected', u'refrigerator', u'forward', u'analyze', u'slight', u'record', u'manufacture', u'mountains', u'bond', u'elsewhere', u'inches', u'equatorial', u'potassium', u'retina', u'chromatography', u'overheated', u'monomers', u'properly', u'link', u'pacific', u'reef', u'atom', u'coldest', u'prophase', u'migrating', u'perspires', u'dull', u'intestines', u'observation', u'directed', u'perspiration', u'evident', u'microorganisms', u'consist', u'circuits', u'characteristic', u'transmits', u'uv', u'deduce', u'lifespan', u'theoretical', u'planet', u'mobile', u'minamata', u'exploration', u'xyza', u'mature', u'planes', u'continuously', u'z', u'strands', u'called', u'rutherfords', u'retroviruses', u'storing', u'units', u'associated', u'adults', u'acidity', u'defined', u'snapping', u'decreases', u'sepia', u'glossopteris', u'metals', u'inferred', u'globally', u'engineering', u'cure', u'reservoirs', u'behavior', u'ecological', u'amandas', u'kidney', u'defines', u'gametes', u'rainy', u'ancestors', u'amounts', u'fixing', u'pat', u'uranium', u'preventing', u'grouped', u'excrete', u'coded', u'angstrom', u'trait', u'shallower', u'amphibian', u'sails', u'explaining', u'elements', u'titles', u'generates', u'rabies', u'provided', u'endocrine', u'mediumsized', u'kt', u'problems', u'robots', u'visits', u'kindergartener', u'glomerular', u'chirp', u'drag', u'allowing', u'absorbing', u'weigh', u'prepares', u'genetics', u'sides', u'structure', u'ago', u'walcott', u'suffers', u'implies', u'percentages', u'platetectonic', u'electromagnet', u'changeschemical', u'physics', u'droughtresistant', u'confirm', u'holes', u'orbit', u'walked', u'tradeoff', u'depth', u'nasa', u'heats', u'presenting', u'offspring', u'fresh', u'crowbar', u'requires', u'garrett', u'initiating', u'evenly', u'essay', u'gamete', u'code', u'illustrates', u'melts', u'communitys', u'uniformitarianism', u'results', u'astronauts', u'illustrated', u'chewable', u'weatherman', u'lamps', u'shone', u'winds', u'humid', u'moon', u'contributions', u'centers', u'toni', u'cody', u'compact', u'copernicus', u'compose', u'exist', u'meteoroids', u'similar', u'beluga', u'concerned', u'fenced', u'eukaryote', u'young', u'suits', u'granular', u'compost', u'ripened', u'helps', u'stable', u'include', u'dramatically', u'sent', u'mutation', u'infecting', u'star', u'lanthanides', u'provides', u'susceptible', u'listed', u'continues', u'calciumfortified', u'landforms', u'developmentally', u'wave', u'snowshoe', u'windshield', u'spoon', u'dirt', u'cellular', u'mixing', u'animals', u'drinking', u'continued', u'minerals', u'categories', u'northwestern', u'mantle', u'transmission', u'positions', u'notes', u'endosymbiotic', u'colors', u'fewer', u'try', u'threat', u'composting', u'race', u'limestone', u'kit', u'avoiding', u'businesses', u'sensory', u'solutions', u'concluded', u'slow', u'encloses', u'smaller', u'orchard', u'pledge', u'planaria', u'bleeding', u'frost', u'sensation', u'crop', u'rivers', u'aquifer', u'fold', u'noise', u'traveling', u'picked', u'reflected', u'growing', u'acid', u'odd', u'elisa', u'victor', u'universities', u'hourly', u'formats', u'plays', u'business', u'apparatus', u'shivers', u'salivary', u'vampire', u'hereditary', u'access', u'malfunctioning', u'indian', u'twins', u'crustalplate', u'flowing', u'crickets', u'eohippus', u'bird', u'exercise', u'body', u'seasonwhy', u'focuses', u'shoreline', u'filings', u'degree', u'leg', u'respectively', u'gathered', u'pushing', u'commercial', u'giant', u'myosin', u'youngest', u'radicals', u'explore', u'northeast', u'focused', u'sink', u'separation', u'others', u'consideration', u'fifteen', u'vinegar', u'great', u'engage', u'respiration', u'pressure', u'receive', u'involved', u'dispensers', u'mimicry', u'screech', u'calvin', u'leaving', u'engine', u'suggests', u'recently', u'garden', u'resulting', u'insulin', u'assistant', u'difficulty', u'popcorn', u'contaminants', u'residents', u'gene', u'membranebound', u'involves', u'sheila', u'examining', u'regulations', u'insecticides', u'depths', u'apple', u'scales', u'egypt', u'manage', u'kuiper', u'increased', u'outcomes', u'diversion', u'galpagos', u'antibiotics', u'decrease', u'biome', u'motor', u'buildup', u'apply', u'tools', u'cloud', u'chromosomes', u'nest', u'use', u'andreas', u'lens', u'figure', u'consumption', u'remains', u'breakup', u'gregor', u'attracting', u'inquirybased', u'flights', u'frog', u'berries', u'predict', u'circumstances', u'camera', u'beaches', u'vehicle', u'timetable', u'eagles', u'bonds', u'stage', u'chloride', u'automobile', u'deepest', u'gained', u'started', u'tennis', u'tons', u'local', u'humus', u'occurred', u'visibility', u'bodily', u'biomass', u'emigration', u'proton', u'cookware', u'parrots', u'talks', u'nonreactive', u'brochure', u'rare', u'departure', u'unequal', u'pollutant', u'carried', u'parachute', u'account', u'balls', u'ironrich', u'embryos', u'f', u'tunnel', u'plan', u'athlete', u'pour', u'pedaled', u'kristens', u'crossed', u'thin', u'silt', u'universe', u'meet', u'primitive', u'drops', u'dissolve', u'performed', u'control', u'boulder', u'wishes', u'weaker', u'tap', u'bent', u'thirsty', u'escapes', u'process', u'achromatopsia', u'bananas', u'purposes', u'hyphae', u'pieces', u'escaped', u'professor', u'reserves', u'pulling', u'bend', u'slip', u'pelvis', u'bones', u'transpires', u'shrimp', u'onions', u'gains', u'dominant', u'organelle', u'united', u'colonization', u'varied', u'destroying', u'hydrogen', u'bungee', u'six', u'stewardship', u'formations', u'lithobates', u'regions', u'cycads', u'varies', u'forest', u'nonbiased', u'elliptical', u'instead', u'tomatoes', u'zone', u'aquaculture', u'profile', u'haploid', u'bioluminescence', u'express', u'blocks', u'cosmic']

In [21]:
#print keywordlist[4200:]
keywordlist4 = [u'farm', u'stop', u'waters', u'came', u'umbilical', u'concentrations', u'discomfort', u'curving', u'distances', u'meiosis', u'recognizes', u'fits', u'buries', u'wrens', u'syndrome', u'tomato', u'realized', u'revising', u'condors', u'famous', u'light', u'lines', u'microbial', u'element', u'amongst', u'eurasian', u'realizes', u'allow', u'buoyant', u'american', u'opinion', u'counted', u'longest', u'digested', u'highaltitude', u'reversal', u'producer', u'kristen', u'phalanges', u'manufactured', u'despite', u'equally', u'meter', u'interpreted', u'sperm', u'boils', u'ovaries', u'relative', u'including', u'looks', u'methods', u'modified', u'carnivores', u'agricultural', u'freezes', u'galaxy', u'chemistry', u'underwater', u'outer', u'burning', u'located', u'excretory', u'masses', u'dispose', u'anaerobic', u'age', u'permanent', u'degrees', u'spoiling', u'insulator', u'orange', u'clusters', u'health', u'spreading', u'attachment', u'dad', u'designs', u'pure', u'parachutist', u'leann', u'surrounds', u'dam', u'practice', u'molecule', u'runs', u'ship', u'topsoil', u'detergents', u'billiard', u'snake', u'hands', u'front', u'acidic', u'seas', u'realize', u'tankful', u'competes', u'articles', u'february', u'fuses', u'toes', u'radiant', u'traced', u'identified', u'boots', u'pathway', u'lifetime', u'sufficiently', u'slide', u'weather', u'transmitted', u'trap', u'shortage', u'crossing', u'rising', u'savannahs', u'circular', u'experienced', u'upward', u'homogeneous', u'watery', u'hookworms', u'centuries', u'buildings', u'related', u'phloem', u'kitchen', u'society', u'frequency', u'static', u'doubled', u'astronomer', u'measure', u'separating', u'margins', u'sexual', u'vacuoles', u'category', u'prokaryotic', u'house', u'pandemics', u'closer', u'overproduction', u'weightlifters', u'venom', u'rise', u'produced', u'max', u'endospore', u'thriving', u'adaptive', u'supports', u'announced', u'common', u'harold', u'jessica', u'interactions', u'spot', u'reacts', u'maintains', u'galaxies', u'landslides', u'inhaling', u'attending', u'completely', u'york', u'autotrophic', u'ridge', u'pasteur', u'likelihood', u'sing', u'shortest', u'carbohydrates', u'repairing', u'hubble', u'fertility', u'yard', u'capture', u'spill', u'biosphere', u'determining', u'latitudes', u'shells', u'ask', u'keep', u'condense', u'length', u'neutron', u'meals', u'retain', u'gather', u'dextrin', u'stimulate', u'nutrition', u'borealis', u'bilayer', u'matthias', u'volcanic', u'describes', u'cleveland', u'disproved', u'transports', u'reaches', u'echo', u'date', u'improvements', u'spots', u'boats', u'seashells', u'exchange', u'reached', u'quality', u'organ', u'subducted', u'long', u'bracket', u'nebulae', u'los', u'bears', u'revealed', u'winter', u'behaves', u'pots', u'system', u'retreats', u'attack', u'opossum', u'appalachian', u'ankle', u'really', u'radium', u'boil', u'prone', u'fluid', u'temperate', u'hydrosphere', u'stomach', u'sealed', u'mariana', u'shelf', u'explanation', u'shallow', u'reactants', u'stresses', u'lists', u'environmental', u'argon', u'photoperiod', u'simulate', u'branches', u'july', u'resistance', u'chemosynthetic', u'updating', u'steel', u'unexpectedly', u'photograph', u'viscous', u'blew', u'culture', u'isolation', u'distinguishes', u'infectious', u'limiting', u'claim', u'ss', u'patients', u'since', u'reproduction', u'bind', u'visited', u'negatively', u'providing', u'steep', u'itchy', u'exhibit', u'false', u'kmh', u'clip', u'survives', u'belowstudent', u'robot', u'tide', u'kepler', u'ultrasonic', u'unlikely', u'outstrip', u'close', u'need', u'facing', u'turn', u'demonstration', u'bark', u'viewed', u'continents', u'acceleration', u'dehydrated', u'angle', u'rearranged', u'dishes', u'inhabiting', u'expertreviewed', u'selective', u'able', u'switch', u'parks', u'compacted', u'hive', u'mix', u'hardshelled', u'superviruses', u'stimulus', u'reproducing', u'soap', u'vegetation', u'blades', u'firefly', u'fuels', u'led', u'limb', u'crawling', u'techniques', u'sandy', u'harmful', u'petals', u'irregular', u'combustion', u'connected', u'hydrochloric', u'recrystallized', u'tanaka', u'vultures', u'nucleotide', u'neptune', u'fair', u'vitamins', u'philosophers', u'topographic', u'placement', u'consequences', u'bred', u'request', u'disease', u'brought', u'face', u'graphs', u'construction', u'error', u'mechanical', u'lakes', u'built', u'highest', u'normally', u'logical', u'fact', u'gain', u'deletion', u'atmosphere', u'selection', u'survived', u'bulging', u'chances', u'rainbows', u'stove', u'charles', u'titled', u'forelimbs', u'multicellular', u'bring', u'evolutionary', u'greenhouse', u'telling', u'taxonomic', u'chicken', u'savanna', u'technologies', u'shield', u'one', u'nearer', u'coloring', u'based', u'estuary', u'plus', u'fleshy', u'jar', u'brighten', u'hydroelectric', u'eat', u'surroundings', u'planted', u'molecules', u'inherited', u'fuel', u'riding', u'experiencing', u'b', u'neuron', u'astronomical', u'achieve', u'discovers', u'sulfide', u'mesozoic', u'injuries', u'cube', u'listened', u'starch', u'sprouting', u'convert', u'rolling', u'rhino', u'background', u'reinvestigated', u'first', u'vehicles', u'mgs', u'beam', u'ones', u'arrangement', u'joins', u'exchanges', u'dodo', u'penetrate', u'chips', u'pharmaceutical', u'antagonist', u'areas', u'crabs', u'gray', u'processes', u'toed', u'initial', u'trucks', u'brain', u'encounters', u'cycled', u'state', u'scientist', u'teams', u'influenced', u'jaime', u'exercises', u'processed', u'contain', u'cleavage', u'ohio', u'fixed', u'asian', u'sediment', u'generations', u'view', u'conversions', u'available', u'shedding', u'temporarily', u'constructing', u'exists', u'spectrometers', u'hardwood', u'variations', u'exerted', u'pitch', u'humans', u'determination', u'unless', u'operates', u'breathe', u'intensity', u'officials', u'powder', u'violet', u'plasticwrapped', u'endothermic', u'dnas', u'packed', u'situation', u'destroyed', u'phases', u'brittle', u'variationsred', u'attempting', u'pattern', u'complementary', u'southward', u'granite', u'tend', u'xray', u'written', u'investigators', u'closed', u'membrane', u'crude', u'nuclear', u'boundary', u'contaminated', u'kidneys', u'mainly', u'ends', u'converting', u'ability', u'horizontally', u'importance', u'mendeleevs', u'emitted', u'deliver', u'day', u'job', u'hypothesis', u'key', u'nicolaus', u'interchangeable', u'psychrometer', u'componentswhich', u'problem', u'declining', u'david', u'temples', u'observable', u'defense', u'sediments', u'always', u'freely', u'climates', u'decay', u'ramp', u'emulsify', u'boulders', u'trips', u'fulfilled', u'etc', u'harvard', u'grain', u'cover', u'plankton', u'catastrophic', u'swallow', u'safely', u'foodquickgrow', u'aquarium', u'photographs', u'torn', u'co', u'author', u'cm', u'cl', u'wall', u'ca', u'doglike', u'coexistence', u'environments', u'walk', u'packaging', u'distinguish', u'heredity', u'celestial', u'twoweek', u'deflected', u'table', u'allocated', u'crystalline', u'rpm', u'carnivorous', u'nose', u'april', u'strand', u'reproduces', u'cavities', u'powered', u'tremendous', u'copies', u'offered', u'immense', u'genetic', u'poured', u'slowly', u'chalkboard', u'marsh', u'proposal', u'absorbent', u'swiftmoving', u'choose', u'antlers', u'bce', u'tissues', u'diversity', u'painted', u'moved', u'sufficient', u'listing', u'protect', u'latitude', u'finished', u'cubes', u'roasting', u'covered', u'kicks', u'last', u'fellow', u'incompleteness', u'present', u'journals', u'homes', u'factual', u'laser', u'abandoned', u'unlike', u'wilt', u'plain', u'align', u'harder', u'value', u'choices', u'harden', u'surveyors', u'hovering', u'reliability', u'fault', u'wild', u'medicinal', u'tommy', u'exerts', u'discoveries', u'referees', u'comets', u'francie', u'examines', u'digesting', u'home', u'almost', u'molecular', u'blood', u'squid', u'thus', u'site', u'surface', u'indoaustralian', u'meteoroid', u'watched', u'supergrow', u'convection', u'foreign', u'conserve', u'greater', u'proxima', u'let', u'heterotroph', u'jaws', u'balloon', u'radiation', u'cross', u'eric', u'became', u'northwest', u'mineralrich', u'bacterialike', u'outbreak', u'foxs', u'cream', u'infer', u'blooms', u'difficult', u'injured', u'tree', u'responses', u'material', u'energyrich', u'planting', u'absorb', u'reporting', u'disagree', u'tetrapods', u'forecasts', u'ion', u'dogma', u'taste', u'dust', u'pedal', u'sunburn', u'magician', u'warming', u'diabetes', u'south', u'reflection', u'disrupted', u'requirements', u'disappears', u'plasma', u'well', u'fever', u'antonio', u'thought', u'patterns', u'catastrophically', u'katelyn', u'applying', u'hazard', u'position', u'uses', u'muscle', u'sensing', u'drawing', u'usual', u'spherical', u'load', u'taller', u'coaster', u'enormously', u'trophic', u'less', u'percent', u'accurate', u'identically', u'means', u'replenish', u'sources', u'bivalves', u'treeless', u'distant', u'surat', u'predicts', u'kernels', u'joining', u'paul', u'housing', u'studying', u'rotation', u'thaliana', u'consume', u'flashlight', u'overuse', u'rapid', u'ferns', u'panther', u'satellites', u'absorption', u'antarctic', u'point', u'overhunted', u'lake', u'discuss', u'coneshaped', u'battery', u'add', u'combine', u'developments', u'past', u'stratum', u'usage', u'sixtyfive', u'matched', u'earthquakes', u'reformed', u'details', u'identical', u'tests', u'recycling', u'quarks', u'fate', u'worlds', u'checking', u'biologists', u'gasolinepowered', u'inversion', u'tigers', u'fats', u'wool', u'increases', u'sparrows', u'five', u'know', u'denser', u'propeller', u'immediately', u'intersection', u'cups', u'prominent', u'placenta', u'subducting', u'displaced', u'tail', u'necessary', u'like', u'onion', u'sizes', u'safest', u'perpetual', u'canyon', u'solutes', u'loses', u'alkali', u'girl', u'longterm', u'backbones', u'highspeed', u'subdivided', u'ribosomes', u'nonfunctional', u'sugarwater', u'troposphere', u'works', u'semimetal', u'stalagmites', u'bowl', u'gaining', u'shed', u'unable', u'uplifted', u'habitat', u'sequence', u'sustained', u'phenomena', u'searching', u'library', u'mosquitoes', u'hair', u'daylight', u'growth', u'warmer', u'scatters', u'credited', u'proper', u'small', u'retention', u'highenergy', u'recognition', u'planetesimal', u'happens', u'leaf', u'colonized', u'lead', u'demonstrates', u'avoid', u'thunderstorms', u'hurricane', u'high', u'philosopher', u'detergent', u'magnitude', u'threebase', u'inflammation', u'economical', u'rockets', u'landscape', u'chains', u'tides', u'demonstrated', u'locate', u'biology', u'blowing', u'coated', u'conveyer', u'exposure', u'reaching', u'journal', u'bowling', u'daffodils', u'expansion', u'peas', u'paper', u'unevenly', u'host', u'circumstance', u'although', u'occupied', u'nail', u'fibrosis', u'beaks', u'beneath', u'counties', u'publishing', u'beaker', u'sister', u'quills', u'incline', u'kevin', u'getting', u'column', u'freedom', u'spilled', u'clinical', u'neurons', u'foil', u'recognized', u'angeles', u'andes', u'seeds', u'sweat', u'cycles', u'cranberry', u'trials', u'insufficient', u'tongue', u'bloodstream', u'microfilaments', u'create', u'revise', u'massachusetts', u'owl', u'photovoltaic', u'contractions', u'bulb', u'suited', u'keeps', u'penlight', u'zones', u'nerves', u'washington', u'functional', u'billions', u'mushrooms', u'included', u'camouflaged', u'congregate', u'fertilizer', u'brush', u'female', u'speciation', u'amylase', u'interaction', u'infection', u'descriptions', u'follows', u'fat', u'carbon', u'additional', u'synthesized', u'transfer', u'vegetables', u'moderate', u'spiral', u'basins', u'noticed', u'continental', u'spread', u'elaphe', u'development', u'inner', u'resists', u'obstacle', u'notices', u'breeding', u'presently', u'calculations', u'fails', u'function', u'margaret', u'cricket', u'brittany', u'north', u'doublestranded', u'springfed', u'constantly', u'brand', u'hundreds', u'attributed', u'volume', u'reminds', u'repeated', u'lowered', u'plague', u'goals', u'nut', u'muons', u'technician', u'moseleys', u'ear', u'hg', u'genetically', u'partially', u'count', u'also', u'fragmentation', u'colorings', u'baobab', u'converter', u'places', u'whether', u'sending', u'gravitational', u'daffodil', u'smooth', u'bluefooted', u'excitement', u'placed', u'books', u'dangerous', u'lynx', u'cake', u'demonstrate', u'distribution', u'piece', u'arno', u'display', u'strengthens', u'densehow', u'block', u'acids', u'rabbits', u'differentiated', u'opposing', u'resting', u'universal', u'year', u'condition', u'pie', u'fertilizers', u'carbohydrate', u'consumed', u'beats', u'referee', u'periods', u'aristotle', u'ink', u'particulate', u'crystals', u'proceeds', u'chromosome', u'betelgeuse', u'windgenerated', u'functions', u'compared', u'improving', u'loud', u'variety', u'pumped', u'corporation', u'oak', u'aquariums', u'innermost', u'vacuole', u'spraying', u'book', u'compares', u'special', u'normal', u'droplets', u'chemosynthesis', u'overpopulation', u'wing', u'flows', u'chemist', u'conclusion', u'maximizes', u'repeat', u'kinds', u'topic', u'waves', u'emerges', u'exothermic', u'june', u'vacuumsealed', u'variation', u'stay', u'pennsylvania', u'chance', u'medications', u'pumps', u'shrank', u'brighter', u'airtight', u'barometric', u'seismic', u'astronomy', u'friends', u'shown', u'phosphorus', u'searches', u'constant', u'stirred', u'breaks', u'spores', u'fertilize', u'supermassive', u'factors', u'asymmetrical', u'rule', u'original', u'hiker', u'amanda', u'portion', u'status', u'volumes', u'tilted', u'pump', u'lives', u'gardens', u'grocery', u'monomer']



In [22]:
keywordlist = keywordlist1
output_save_path = '/Users/MK/Desktop/Minchul/Machine Learning/Text Mining/get_wikipedia_contents_module_all_ck_data'
get_wikipedia_content_based_on_keyword_list_one_file_per_keyword(keywordlist, output_save_path)

0 1400 0.0 xylem
1 1400 0.000714285714286 limited
DisambiguationError limited
2 1400 0.00142857142857 similarity
DisambiguationError similarity
3 1400 0.00214285714286 phenotypical
4 1400 0.00285714285714 alleles
5 1400 0.00357142857143 magnetic
6 1400 0.00428571428571 saves
DisambiguationError saves
7 1400 0.005 desirable
DisambiguationError desirable
8 1400 0.00571428571429 crumpled
9 1400 0.00642857142857 yellow
10 1400 0.00714285714286 chaos
DisambiguationError chaos
11 1400 0.00785714285714 four
12 1400 0.00857142857143 tensile
13 1400 0.00928571428571 heliocentric
14 1400 0.01 controversial
15 1400 0.0107142857143 consists
DisambiguationError consists
16 1400 0.0114285714286 oldest
DisambiguationError oldest
17 1400 0.0121428571429 worked
DisambiguationError worked
18 1400 0.0128571428571 aggression
19 1400 0.0135714285714 poorly
20 1400 0.0142857142857 relationships
DisambiguationError relationships
21 1400 0.015 whose
22 1400 0.0157142857143 fronts
DisambiguationError fronts
23

/Users/MK/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


TypeError: file() argument 1 must be encoded string without NULL bytes, not unicode

In [28]:
keywordlist = keywordlist1[1368:]
output_save_path = '/Users/MK/Desktop/Minchul/Machine Learning/Text Mining/get_wikipedia_contents_module_all_ck_data'
get_wikipedia_content_based_on_keyword_list_one_file_per_keyword(keywordlist, output_save_path)

0 32 0.0 baseball
1 32 0.03125 pasteurization
2 32 0.0625 yards
3 32 0.09375 kilometers
4 32 0.125 goldfish
5 32 0.15625 along
DisambiguationError along
6 32 0.1875 constructive
7 32 0.21875 teacher
8 32 0.25 change
DisambiguationError change
9 32 0.28125 knowledge
10 32 0.3125 box
11 32 0.34375 passengers
12 32 0.375 unidentified
13 32 0.40625 incoming
14 32 0.4375 canadian
15 32 0.46875 healthy
16 32 0.5 shift
DisambiguationError shift
17 32 0.53125 studied
DisambiguationError studied
18 32 0.5625 funded
19 32 0.59375 mendeleev
20 32 0.625 commonly
DisambiguationError commonly
21 32 0.65625 somatic
DisambiguationError somatic
22 32 0.6875 spheres
23 32 0.71875 accomplished
DisambiguationError accomplished
24 32 0.75 franklin
DisambiguationError franklin
25 32 0.78125 usually
26 32 0.8125 diploid
27 32 0.84375 noticeably
28 32 0.875 studies
DisambiguationError studies
29 32 0.90625       @                                                                                                

1400